In [1]:
import os
from pathlib import Path
import random
import time

random_state = random.randint(1,999)
display(random_state)

# COnfiguraciones para tensorflow
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

417

In [ ]:
import kaggle